# Trabajo Practico 3: Machine Learning

integrantes: Alvarez Vico Jazmín

             Cortés Conde Titó Javier María
             
             Frassia Fernando 

In [6]:
import scipy as sp
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import pylab
from scipy.signal import welch
import scipy.io
import os
from math import log
log2= lambda x:log(x,2)
from collections import defaultdict

def dar_bandas(lista_sujetos, lista_electrodos):
    elect = lista_sujetos[0][:,1,:]
    welch = sp.signal.welch(elect, noverlap=100,fs= 250, nperseg = 201)
    index = (np.abs(welch[0]-50)).argmin()
    
    df_alphas = pd.DataFrame()
    df_betas = pd.DataFrame()
    df_thetas = pd.DataFrame()
    df_deltas = pd.DataFrame()
    df_gamas = pd.DataFrame()

    i=1
    for sujeto in lista_sujetos:

        potencias_media = []
        for count in lista_electrodos:
            welch = sp.signal.welch(sujeto[:,count - 1,:], noverlap=100, fs=250, nperseg = 201)
            frecuencias = welch[0][:index]
            potencias = welch[1][:, :index]
            promedio_potencias = potencias.mean(axis=0)
            potencias_media.append(promedio_potencias)

        potencias_media = np.array(potencias_media)
        medias = potencias_media.mean(axis=0)

        index_delta = (np.abs(frecuencias-4)).argmin()
        delta = medias[:index_delta]
        df_deltas['sujeto'+str(i)] = delta

        index_theta = (np.abs(frecuencias-8)).argmin()
        theta= medias[index_delta+1:index_theta]
        df_thetas['sujeto'+str(i)] = theta

        index_alpha = (np.abs(frecuencias-13)).argmin()
        alpha = medias[index_theta+1:index_alpha]
        df_alphas['sujeto'+str(i)] = alpha

        index_beta = (np.abs(frecuencias-30)).argmin()
        beta = medias[index_alpha+1:index_beta]
        df_betas['sujeto'+str(i)] = beta

        gama = medias[index_beta+1:]
        df_gamas['sujeto'+str(i)] = gama
        i+=1
    
    return (df_alphas, df_betas, df_deltas, df_gamas, df_thetas)

def dar_potXbandas(lista_sujetos, lista_electrodos):
    df_alphas = pd.DataFrame()
    df_betas = pd.DataFrame()
    df_thetas = pd.DataFrame()
    df_deltas = pd.DataFrame()
    df_gamas = pd.DataFrame()
    
    df_alphas, df_betas, df_deltas, df_gamas, df_thetas = dar_bandas(lista_sujetos, lista_electrodos)
    
    potenciasXbanda = pd.DataFrame()
    
    sum_alpha = df_alphas.sum()
    potenciasXbanda['alpha'] = sum_alpha
    sum_beta = df_betas.sum()
    potenciasXbanda['beta'] = sum_beta
    sum_delta = df_deltas.sum()
    potenciasXbanda['delta'] = sum_delta
    sum_gama = df_gamas.sum()
    potenciasXbanda['gama'] = sum_gama
    sum_theta = df_thetas.sum()
    potenciasXbanda['theta'] = sum_theta
    
    factor_normalizacion = sum_alpha+sum_beta+sum_gama+sum_theta + sum_delta 
    
    potenciasXbanda['alpha_norm'] = sum_alpha/factor_normalizacion
    potenciasXbanda['beta_norm'] = sum_beta/factor_normalizacion
    potenciasXbanda['delta_norm'] = sum_delta/factor_normalizacion
    potenciasXbanda['gama_norm'] = sum_gama/factor_normalizacion
    potenciasXbanda['theta_norm'] = sum_theta/factor_normalizacion
    
    return potenciasXbanda




In [72]:
def promediar_epoch(sujeto, lista_electrodo):
    promedio_electrodos = pd.DataFrame()
    for electrodo in lista_electrodo:
        df = pd.DataFrame(sujeto[:,electrodo-1,:])
        df_Media = df.mean(axis=0)
        promedio_electrodos = df_Media
        #print(promedio_electrodos)
        #print(df_Media)
    
    promedio_electrodos = promedio_electrodos/len(lista_electrodo)
    return promedio_electrodos

def eleccion_n(x):
    res = (x.max()-x.min())/(3.5*x.std()*len(x)**(-1/3))
    return int(res)

def entropia(sujeto, lista_electrodo):
    x = promediar_epoch(sujeto, lista_electrodo)
    bins = eleccion_n(x)
    counts_x, bins = np.histogram(x, bins=bins)
    return sp.stats.entropy(counts_x)

def entropia_sujetos(lista_sujetos, lista_electrodo):
    aux = []
    for i in range(len(lista_sujetos)):
        entro = entropia(lista_sujetos[i], lista_electrodo)
        aux.append(entro)
    
    df_entropia = pd.DataFrame(data = aux,index = ['sujeto'+str(i+1) for i in range(len(lista_sujetos))], columns = ['entropia'])
    return df_entropia


def informacion_mutua(sujeto, lista_electrodo1, lista_electrodo2):
    x = promediar_epoch(sujeto, lista_electrodo1)
    y = promediar_epoch(sujeto, lista_electrodo2)
    bins_x = eleccion_n(x)
    bins_y = eleccion_n(y)
    bins = min(bins_x, bins_y)
    counts_xy, bins_x, bins_y = np.histogram2d(x, y, bins=(bins, bins))
    counts_x, bins = np.histogram(x, bins=bins)
    counts_y, bins = np.histogram(y, bins=bins)

    counts_xy += 1
    counts_x += 1
    counts_y += 1
    P_xy = counts_xy / np.sum(counts_xy, dtype=float)
    P_x = counts_x / np.sum(counts_x, dtype=float)
    P_y = counts_y / np.sum(counts_y, dtype=float)

    I_xy = np.sum(P_xy * np.log2(P_xy / (P_x.reshape(-1, 1) * P_y)))

    return I_xy / (sp.stats.entropy(counts_x) + sp.stats.entropy(counts_y))

def informacion_mutua_sujetos(lista_sujetos, lista_electrodo1, lista_electrodo2):
    aux = []
    for i in range(len(lista_sujetos)):
        entro = informacion_mutua(lista_sujetos[i], lista_electrodo1, lista_electrodo2)
        aux.append(entro)
    
    df = pd.DataFrame(data = aux,index = ['sujeto'+str(i+1) for i in range(len(lista_sujetos))], columns = ['informacion_mutua'])
    return df

def crear_features(lista_sujetos, lista_electrodos1, lista_electrodos2):
    matriz_informacion = dar_potXbandas(lista_sujetos, lista_electrodos1)
    matriz_informacion['entropia'] =  entropia_sujetos(lista_sujetos, lista_electrodos1)
    matriz_informacion['informacion_mutua'] = informacion_mutua_sujetos(lista_sujetos, lista_electrodos1, lista_electrodos2)
    
    return matriz_informacion

In [2]:
lista_sujetosP = []
matrices = os.listdir('../tp2/matricesP/')
for mat in matrices:
    m = sp.io.loadmat('../tp2/matricesP/' + mat)
    mm = m['data']
    lista_sujetosP.append(mm)
    
       

lista_sujetosS = []
matrices = os.listdir('../tp2/matricesS/')
for mat in matrices:
    m = sp.io.loadmat('../tp2/matricesS/' + mat)
    mm = m['data']
    lista_sujetosS.append(mm)
    
    
    

In [78]:
lista_electrodos = [8,44,80,131,185]
lista_electrodos_complemento = [15, 42, 78, 154, 206]

matP = crear_features(lista_sujetosP, lista_electrodos, lista_electrodos_complemento)

matS = crear_features(lista_sujetosS, lista_electrodos, lista_electrodos_complemento)
matP


,alpha,beta,delta,gama,theta,alpha_norm,beta_norm,delta_norm,gama_norm,theta_norm,entropia,informacion_mutua
sujeto1,1.643789e-12,1.500178e-12,3.346648e-11,7.367694e-13,5.002184e-12,0.038815,0.035424,0.790247,0.017397,0.118117,1.509751,0.785227
sujeto2,1.386198e-12,8.356118e-13,3.973294e-11,1.943338e-13,5.869616e-12,0.028868,0.017402,0.827447,0.004047,0.122236,1.669884,0.954295
sujeto3,2.079876e-12,9.704867e-13,1.800538e-11,1.959763e-13,5.330394e-12,0.078243,0.036509,0.677349,0.007372,0.200526,1.743208,0.607381
sujeto4,9.192067e-13,1.217792e-12,1.578574e-11,3.804462e-13,2.774671e-12,0.043610,0.057776,0.748925,0.018050,0.131639,1.694049,0.393337
sujeto5,7.001011e-13,1.198941e-12,1.688559e-11,5.792357e-13,5.020994e-12,0.028710,0.049167,0.692462,0.023754,0.205906,1.707549,0.317960
sujeto6,8.097510e-13,9.040271e-13,2.515836e-11,4.155608e-13,5.760429e-12,0.024502,0.027355,0.761264,0.012574,0.174304,1.739647,0.425857
sujeto7,1.382327e-12,1.537592e-12,1.691955e-11,4.819727e-13,7.847380e-12,0.049073,0.054585,0.600648,0.017110,0.278584,1.583128,0.168415
sujeto8,2.611314e-12,1.284137e-12,2.225274e-11,1.491643e-13,2.892218e-12,0.089461,0.043993,0.762352,0.005110,0.099084,1.633277,0.302939


In [79]:
matS

,alpha,beta,delta,gama,theta,alpha_norm,beta_norm,delta_norm,gama_norm,theta_norm,entropia,informacion_mutua
sujeto1,2.139868e-12,2.906016e-12,8.018336e-12,2.326663e-12,2.682131e-12,0.118401,0.160793,0.443663,0.128737,0.148405,1.686995,0.400126
sujeto2,5.151727e-12,2.233744e-12,2.939842e-12,1.154041e-12,7.465811e-13,0.421377,0.182705,0.240459,0.094393,0.061065,1.761392,0.661761
sujeto3,1.152959e-11,4.020837e-12,9.477273e-12,3.153248e-13,5.805119e-12,0.370153,0.129088,0.304264,0.010123,0.186371,1.690017,0.354859
sujeto4,1.755314e-12,4.727181e-12,7.279762e-12,4.720905e-12,1.894635e-12,0.086139,0.231977,0.357240,0.231669,0.092975,1.784680,0.198473
sujeto5,2.430889e-12,2.597884e-12,4.310585e-12,3.423912e-12,2.108045e-12,0.163462,0.174691,0.289859,0.230236,0.141752,1.777411,0.270649
sujeto6,6.480402e-12,4.667152e-12,1.078479e-11,3.347743e-12,2.203755e-12,0.235790,0.169814,0.392405,0.121808,0.080184,1.564652,1.183961
sujeto7,6.586528e-12,2.562401e-12,2.780564e-12,1.097578e-12,1.208275e-12,0.462688,0.180003,0.195328,0.077102,0.084879,1.690656,0.664184
sujeto8,1.219421e-11,2.150920e-12,3.536900e-12,2.245349e-13,1.558069e-12,0.620109,0.109380,0.179861,0.011418,0.079232,1.528385,0.338487
sujeto9,7.224353e-12,1.933603e-12,3.712084e-12,1.806182e-13,1.501185e-12,0.496456,0.132877,0.255094,0.012412,0.103161,1.790704,0.317733
